# Documenting sample selection (part 2)
<!-- > "Using decorator to document sample selection process." -->

- hide:true
- toc: true
- badges: true
- comments: true
- categories: [python]

In [1]:
from collections import Counter, OrderedDict
from functools import wraps

import pandas as pd
import numpy as np

First, build a counter object.

In [12]:
class OrderedCounter(Counter, OrderedDict):
    'Counter that remembers the order elements are first encountered'

    def __repr__(self):
        return '{!s}({!r})'.format(self.__class__.__name__, OrderedDict(self))

    def __reduce__(self):
        return self.__class__, (OrderedDict(self),)
    
OrderedCounter({'a': 1, 'b': 2})

OrderedCounter(OrderedDict([('a', 1), ('b', 2)]))

Method resolution order:

In [3]:
OrderedCounter.__mro__

(__main__.OrderedCounter,
 collections.Counter,
 collections.OrderedDict,
 dict,
 object)

In [58]:
df = pd.read_csv('/Users/fgu/tmp/tab.csv').iloc[:, 1:]
df.head()

,step,counts
0,start-users,283.0
1,start-txs,662580.0
2,start-value,122286176.0
3,drop_noise-users,283.0
4,drop_noise-txs,662580.0


In [59]:
df[['step', 'metric']] = df.step.str.split('-', expand=True)
df.head()

,step,counts,metric
0,start,283.0,users
1,start,662580.0,txs
2,start,122286176.0,value
3,drop_noise,283.0,users
4,drop_noise,662580.0,txs


From here: https://stackoverflow.com/a/44988732/13666841

In [60]:
df = (df.groupby(['step', 'metric'], sort=False)
      .counts.sum()
      .unstack('metric')
      .rename_axis(columns=None)
      .reset_index())
df

,step,users,txs,value
0,start,283.0,662580.0,1.222862e+08
1,drop_noise,283.0,662580.0,1.222862e+08
2,min_number_of_months,239.0,653239.0,1.207088e+08
3,current_account,227.0,636792.0,1.185223e+08
4,min_spend,151.0,439269.0,7.942254e+07
5,min_number_transactions,129.0,384487.0,6.776514e+07
6,diverse_spending,129.0,384487.0,6.776514e+07
7,end,129.0,384487.0,6.776514e+07


In [68]:
# Formatting

df['step'] = df['step'].map(desc)
ints = ['users', 'acc', 'txs']
df[ints] = df[ints].applymap('{:,.0f}'.format)
floats = ['value']
df[floats] = df[floats].applymap('{:,.1f}'.format)

NameError: name 'desc' is not defined

In [62]:
k = df.select_dtypes('number').applymap('{:,.0f}'.format)
k

,users,txs,value
0,283,"662,580","122,286,176"
1,283,"662,580","122,286,173"
2,239,"653,239","120,708,756"
3,227,"636,792","118,522,330"
4,151,"439,269","79,422,540"
5,129,"384,487","67,765,143"
6,129,"384,487","67,765,143"
7,129,"384,487","67,765,143"


In [67]:
df[['value']].applymap(lambda x: x*10)

,value
0,1.222862e+09
1,1.222862e+09
2,1.207088e+09
3,1.185223e+09
4,7.942254e+08
5,6.776514e+08
6,6.776514e+08
7,6.776514e+08


In [ ]:
df.to_latex

In [57]:
df[df.select_dtypes('number').columns] = k
df

,step,users,txs,value
0,start,283,"662,580","122,286,176"
1,drop_noise,283,"662,580","122,286,173"
2,min_number_of_months,239,"653,239","120,708,755"
3,current_account,227,"636,792","118,522,329"
4,min_spend,151,"439,269","79,422,540"
5,min_number_transactions,129,"384,487","67,765,143"
6,diverse_spending,129,"384,487","67,765,143"
7,end,129,"384,487","67,765,143"


In [37]:
fmt = {
    'users': '{:.0f}',
    'txs': '{:.0f}',
    'value': '{:,.0f}',
}
df = df.style.format(fmt)
df

,step,users,txs,value
0,start,283,662580,"122,286,176"
1,drop_noise,283,662580,"122,286,173"
2,min_number_of_months,239,653239,"120,708,756"
3,current_account,227,636792,"118,522,330"
4,min_spend,151,439269,"79,422,540"
5,min_number_transactions,129,384487,"67,765,143"
6,diverse_spending,129,384487,"67,765,143"
7,end,129,384487,"67,765,143"


In [38]:
df = df.select_dtypes('number').astype('str')
df

AttributeError: 'Styler' object has no attribute 'select_dtypes'

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   users   8 non-null      object
 1   txs     8 non-null      object
 2   value   8 non-null      object
dtypes: object(3)
memory usage: 320.0+ bytes


ValueError: Cannot specify ',' with 's'.

In [ ]:
df.to_csv

,step,users,txs,value
0,start,283,662580,"122,286,176"
1,drop_noise,283,662580,"122,286,173"
2,min_number_of_months,239,653239,"120,708,756"
3,current_account,227,636792,"118,522,330"
4,min_spend,151,439269,"79,422,540"
5,min_number_transactions,129,384487,"67,765,143"
6,diverse_spending,129,384487,"67,765,143"
7,end,129,384487,"67,765,143"


In [44]:
df = df.set_index(['step', 'metric'])
df.head()


counts
step       metric             
start      users         283.0
           txs        662580.0
           value   122286176.0
drop_noise users         283.0
           txs        662580.0

In [45]:
df.unstack(1)

counts                     
metric                        txs  users         value
step                                                  
current_account          636792.0  227.0  1.185223e+08
diverse_spending         384487.0  129.0  6.776514e+07
drop_noise               662580.0  283.0  1.222862e+08
end                      384487.0  129.0  6.776514e+07
min_number_of_months     653239.0  239.0  1.207088e+08
min_number_transactions  384487.0  129.0  6.776514e+07
min_spend                439269.0  151.0  7.942254e+07
start                    662580.0  283.0  1.222862e+08

In [41]:
df.columns.droplevel(0)
df.columns.name = None
df.head()

counts                     
metric                     txs  users         value
step                                               
current_account       636792.0  227.0  1.185223e+08
diverse_spending      384487.0  129.0  6.776514e+07
drop_noise            662580.0  283.0  1.222862e+08
end                   384487.0  129.0  6.776514e+07
min_number_of_months  653239.0  239.0  1.207088e+08

In [19]:
dd.drop

AttributeError: 'NoneType' object has no attribute 'drop'

In [11]:
a = OrderedCounter()
a.update(s_a=1, s_b=2, k_a=3, k_b=4)
a.update(s_a=5, s_b=6, k_a=7, k_b=8)
a

OrderedCounter(OrderedDict([('s_a', 6), ('s_b', 8), ('k_a', 10), ('k_b', 12)]))

Turn it into a dataframe.

In [12]:
df = pd.DataFrame(a.items(), columns=['step', 'count'])
df

,step,count
0,s_a,6
1,s_b,8
2,k_a,10
3,k_b,12


Split the step column

In [13]:
df[['step', 'metric']] = df.step.str.split('_', expand=True)
df

,step,count,metric
0,s,6,a
1,s,8,b
2,k,10,a
3,k,12,b


Unstack

In [6]:
df = df.set_index(['step', 'metric']).unstack(1)
df

count    
metric     a   b
step            
k         10  12
s          6   8

Flatten

In [7]:
df.columns = df.columns.droplevel(0)
df

metric,a,b
step,,
k,10,12
s,6,8


In [126]:
df.columns.name = None

In [127]:
df

,a,b
step,,
k,10,12
s,6,8


In [128]:
df.reset_index()

,step,a,b
0,k,10,12
1,s,6,8


## Main sources

- [Fluent Python](https://www.oreilly.com/library/view/fluent-python/9781491946237/)
- [Python Cookbook](https://www.oreilly.com/library/view/python-cookbook-3rd/9781449357337/)
- [Learning Python](https://www.oreilly.com/library/view/learning-python-5th/9781449355722/)